# Classfication with the correct formatted epoch of 500ms
* this notebook will first label the epoch data events as 4: Language or 5 : nonlanguage. Then do classification on the labeled epoch data
* The goal is to be able to classfy the correct data based on language vs. nonlanguage and see what kind of scores we get.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time

In [ ]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [ ]:
epoch=remove_artifacts_and_create_epochs('C:\\USB Drive\\NewEEG-200s\\SA20140205\\20140205_1230.set')

In [ ]:
## Methods for dividing the events into trials with 3 events make up 1 trial
epochs=[[]]
j=0
for i in range(len(epoch.events[:,-1])):
    if i % 3 ==0:
       
        epochs.append([])
        j=j+1
        epochs[j].append(epoch.events[i,-1])
              
    else:
        epochs[j].append(epoch.events[i,-1])
        # Convert the epoch data into list of list
        #Each list contains three event ids as 1 trial

In [ ]:
np.shape(epoch)

In [ ]:
epochs

In [ ]:
epoch.events[2,-1]

In [ ]:
epoch.events[:,-1]

In [ ]:
for events in epochs:
    if  4 in events or 1 in events or 13 in events or 16 in events:
        for i in range(len(events)):
                events[i]=4
    else:
        for i in range(len(events)):
                events[i]=5
        # Label each trial as either Language or non-language

In [ ]:
epochs

In [ ]:
## Put the labeled data back to the epoch event object
epochs.pop(0)
flattened_list = [y for x in epochs for y in x]
for i in range(len(epoch.events)):
    epoch.events[i]=flattened_list[i]

In [ ]:
epoch.plot()

In [ ]:
## knn classifier
def classify_test_KNN(X,y):

    clf = make_pipeline(Vectorizer(),
                        StandardScaler(),
                        KNN(n_neighbors=11))
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
    clf.fit(X_train, y_train)
    
    score = clf.score(X_test, y_test)
    
    return score
epoch.events[:,-1]

In [ ]:
## Methods for convert our events into binary, 4: Language, 5:Non Language
def filter_event_id(epoch):
    epochs=[[]]
    j=0
    for i in range(len(epoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(epoch.events)):
        epoch.events[i]=flattened_list[i]
    return epoch

In [ ]:
## methods for getting the target and features for our data
def get_samples_targets(epochs):
    X = epochs.get_data();
    y = epochs.events[:,-1]
    return X, y

In [ ]:
## Classfication with KNN
X = epoch.get_data()
y=epoch.events[:,-1]

clf = make_pipeline(Vectorizer(),
                        KNN(n_neighbors=13))
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)
print (score)
#classfication with KNN.

In [ ]:
path = 'C:\\USB Drive\\NewEEG-200s'

eeglab_files = os.listdir(path)
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
for f in eeglab_files:
    print(f)

In [ ]:
scores1 = []

path = 'C:\\USB Drive\\NewEEG-200s'
## Run classfication with all the subject files in the path
eeglab_files = os.listdir(path)
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
for f in eeglab_files:
    epoch=remove_artifacts_and_create_epochs(f)
    epoch = filter_event_id(epoch)
    X,y = get_samples_targets(epoch)
    scores1.append([f, classify_test_KNN(X,y)])
    print(scores1)

In [ ]:
epoch=remove_artifacts_and_create_epochs('C:\\USB Drive\\NewEEG-200s\\SA20131216\\20131216_1310.set')

In [ ]:
def filter_event_id(epoch):
    epochs=[[]]
    j=0
    for i in range(len(epoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(epoch.events)):
        epoch.events[i]=flattened_list[i]
    return epoch

In [ ]:
filter_event_id(epoch)

In [ ]:
epoch.events.shape

In [ ]:
# do PCA to reduce dimensions and data needed for classification.
pca = UnsupervisedSpatialFilter(PCA(28), average=False) # PCA, keep 9 components 

epoch_data = epoch.get_data()
pca_data = pca.fit_transform(epoch_data)

In [ ]:
pca_data.shape

In [ ]:
## Get Kurtosis, vairance, skewness for our data
def get_mean_band(data):
    final_variances = []
    for d in data:
    #     print (data.shape)
        variances = []
        skewnesses = []
        kurtosises = []
        stats = []
        for channel in d:
    #         print(channel.shape)
            var = channel.var()
            variances.append(var)
            
            skewness = skew(channel, axis=0)
#             print(skewness)

            skewnesses.append(skewness)
#             print(skewness)
            kurt = kurtosis(channel)
            kurtosises.append(kurt)
#         print(skewnesses)
#         mean_skew = np.mean(skewnesses)
#         stats.append(mean_skew)
#         stats.append(np.mean(variances))
        final_variances.append([np.mean(variances), np.mean(skewnesses), np.mean(kurtosises)])
    
    return np.array(final_variances)

In [ ]:
mean_statisitcs = get_mean_band(pca_data)
epoch.events

In [ ]:
mean_statisitcs.shape
epoch.events.shape

In [ ]:
mean_statisitcs = mean_statisitcs.swapaxes(0,1)

In [ ]:
mean_statisitcs.shape

In [ ]:
# classification pipeline
start = time.time()
clf = make_pipeline(Vectorizer(),
                    StandardScaler(),
                    LogisticRegression(penalty='l1'))
end = time.time()
print("clf elapsed time: {0}".format(end - start))

start_master = time.time()
cv = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)

labels = epoch.events[:,-1]
preds = np.empty(len(mean_statisitcs))
for train, test in cv.split(mean_statisitcs, labels):
    start = time.time()
    clf.fit(mean_statisitcs[train], labels[train])
    preds[test] = clf.predict(mean_statisitcs[test])
    end = time.time()
    print("kfold elapsed time: {0}".format(end - start))
end = time.time()
print("classification elapsed time: {0}".format(end - start_master))


start = time.time()
target_names = ['Audio', 'Visual']
report = classification_report(labels, preds, target_names=target_names)
print(report)
end = time.time()
print("reporting elapsed time: {0}".format(end - start))

In [ ]:
epoch_data = epoch.get_data()
epoch_data.shape

In [ ]:
pca_data.shape

In [ ]:
def get_mean_band(data):
    final_variances = []
    for d in data:
    #     print (data.shape)
        variances = []
        skewnesses = []
        kurtosises = []
        stats = []
        for channel in d:
    #         print(channel.shape)
            var = channel.var()
            variances.append(var)
            
            skewness = skew(channel, axis=0)
#             print(skewness)

            skewnesses.append(skewness)
#             print(skewness)
            kurt = kurtosis(channel)
            kurtosises.append(kurt)
#         print(skewnesses)
#         mean_skew = np.mean(skewnesses)
#         stats.append(mean_skew)
#         stats.append(np.mean(variances))
        final_variances.append([variances, skewnesses, kurtosises])
    
    return np.array(final_variances)

In [ ]:
mean_statisitcs = get_mean_band(pca_data)

In [ ]:
Kurtosis_data = mean_statisitcs.swapaxes(1,2)
Kurtosis_data.shape

In [ ]:
labels =epoch.events[:,-1]
score2 = []
for train,test in cv.split(Kurtosis_data,target_V_vs_A):
    X_train = Kurtosis_data[train] 
    #print("shape of training features(data):",X_train.shape)
    y_train =labels [train] 
    #print("shape of training target:",y_train.shape)
    X_test = Kurtosis_data[test] 
    #print("shape of testing data:",X_test.shape)
    y_test = labels [test]
    #print("shape of testing target:",y_test.shape)
    
    #fit the model to training set
    clf.fit(X_train,y_train)
    
    #returns the mean accuracy on test data and labels
    score2.append(clf.score(X_test,y_test))

In [ ]:
print("mean classification score", np.mean(score2))

In [ ]:
start = time.time()
clf = make_pipeline(
                    Vectorizer(),
                    StandardScaler(),
                    LinearDiscriminantAnalysis())
end = time.time()
print("clf elapsed time: {0}".format(end - start))

start_master = time.time()
cv = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)

labels = epoch.events[:,-1]
preds = np.empty(len(mean_statisitcs))
for train, test in cv.split(mean_statisitcs, labels):
    start = time.time()
    clf.fit(mean_statisitcs[train], labels[train])
    preds[test] = clf.predict(mean_statisitcs[test])
    end = time.time()
    print("kfold elapsed time: {0}".format(end - start))
end = time.time()
print("classification elapsed time: {0}".format(end - start_master))


start = time.time()
target_names = ['Audio', 'Visual']
report = classification_report(labels, preds, target_names=target_names)
print(report)
end = time.time()
print("reporting elapsed time: {0}".format(end - start))

In [ ]:
print("mean classification score", np.mean(score2))

In [ ]:
epoch.events[:,-1]

In [ ]:
epoch_data.shape

In [ ]:
epoch=epoch.drop_channels(['Lm','VEOG','Rm'])

In [ ]:
epoch.get_data=epoch_data

In [ ]:
epoch.get_data.shape

In [ ]:
len(epoch.ch_names)

In [ ]:
norm_pipe = make_pipeline(Vectorizer(), 
                          StandardScaler())
norm_pipe.fit(Kurtosis_data)

In [ ]:
scaled_data = norm_pipe.transform(Kurtosis_data)
scaled_data.shape